In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

/Users/vivekvajipey/miniconda3/envs/reasoning/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [4]:
# model = AutoModelForCausalLM.from_pretrained("mistralai/Mixtral-8x7B-v0.1")
# tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-v0.1")

In [5]:
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

config.json: 100%|██████████| 571/571 [00:00<00:00, 2.54MB/s]
model.safetensors.index.json: 100%|██████████| 25.1k/25.1k [00:00<00:00, 59.4MB/s]
model-00001-of-00002.safetensors: 100%|██████████| 9.94G/9.94G [05:39<00:00, 29.3MB/s]
model-00002-of-00002.safetensors: 100%|██████████| 4.54G/4.54G [04:50<00:00, 15.7MB/s]
tokenizer_config.json: 100%|██████████| 967/967 [00:00<00:00, 1.68MB/s]
tokenizer.model: 100%|██████████| 493k/493k [00:00<00:00, 28.1MB/s]
tokenizer.json: 100%|██████████| 1.80M/1.80M [00:00<00:00, 5.65MB/s]
special_tokens_map.json: 100%|██████████| 72.0/72.0 [00:00<00:00, 209kB/s]


In [7]:
model.to(device)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm(

In [ ]:
import pandas as pd

df = pd.read_csv('data/112_gsm8k_gpt35_cot_responses_autograded.csv')

In [8]:
prompt = "My favourite condiment is"

model_inputs = tokenizer([prompt], return_tensors="pt").to(device)
model.to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
tokenizer.batch_decode(generated_ids)[0]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"<s> My favourite condiment is garlic aioli. I like it with all kinds of food: salad, roast veggies, potato bake, steamed vegetables, chips, seafood. And just about anything else.\n\nI've posted two aioli recipes using egg. Both egg-based aioli and yoghurt based dips are extremely versatile and delicious. But I often want to add a spoonful of aioli to my pasta and serve it to my son. Egg"

In [10]:
text = "Here is some example text"


tensor([[-6.6415, -1.6646, -3.7755, -7.8877, -2.3673]], device='mps:0',
       grad_fn=<LogBackward0>)


In [15]:
def get_logprobs_input(model, tokenizer, text):
    tokens = tokenizer(text, return_tensors="pt").to(device)

    # Getting model outputs
    outputs = model(**tokens, labels=tokens["input_ids"])
    logits = outputs.logits

    # Calculating probabilities and log probabilities for each token
    shift_logits = logits[..., :-1, :].contiguous()
    shift_labels = tokens["input_ids"][..., 1:].contiguous().to(device)
    # Apply softmax to convert logits to probabilities
    probs = torch.nn.functional.softmax(shift_logits, dim=-1)
    # Gather the probabilities of the actual tokens
    actual_probs = torch.gather(probs, -1, shift_labels.unsqueeze(-1)).squeeze(-1)
    # Convert to log probabilities
    log_probs = actual_probs.log()

    for tok, log_probs in zip(tokens[1:], log_probs):
        print(f"{tok}: {log_probs:.6f}")

In [13]:
log_probs.flatten()

tensor([-6.6415, -1.6646, -3.7755, -7.8877, -2.3673], device='mps:0',
       grad_fn=<ViewBackward0>)

Mistral Instruct

In [14]:
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

config.json: 100%|██████████| 596/596 [00:00<00:00, 294kB/s]
model.safetensors.index.json: 100%|██████████| 25.1k/25.1k [00:00<00:00, 27.3MB/s]
model-00001-of-00003.safetensors: 100%|██████████| 4.94G/4.94G [02:49<00:00, 29.2MB/s]
model-00002-of-00003.safetensors: 100%|██████████| 5.00G/5.00G [02:52<00:00, 29.0MB/s]
model-00003-of-00003.safetensors: 100%|██████████| 4.54G/4.54G [02:29<00:00, 30.4MB/s]
generation_config.json: 100%|██████████| 111/111 [00:00<00:00, 799kB/s]
tokenizer_config.json: 100%|██████████| 1.46k/1.46k [00:00<00:00, 15.0MB/s]
tokenizer.model: 100%|██████████| 493k/493k [00:00<00:00, 73.7MB/s]
tokenizer.json: 100%|██████████| 1.80M/1.80M [00:00<00:00, 26.9MB/s]
special_tokens_map.json: 100%|██████████| 72.0/72.0 [00:00<00:00, 252kB/s]


In [16]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm(

In [18]:
def get_logprobs_input(model, tokenizer, text):
    tokens = tokenizer(text, return_tensors="pt").to(device)

    # Getting model outputs
    outputs = model(**tokens, labels=tokens["input_ids"])
    logits = outputs.logits

    # Calculating probabilities and log probabilities for each token
    shift_logits = logits[..., :-1, :].contiguous()
    shift_labels = tokens["input_ids"][..., 1:].contiguous().to(device)
    # Apply softmax to convert logits to probabilities
    probs = torch.nn.functional.softmax(shift_logits, dim=-1)
    # Gather the probabilities of the actual tokens
    actual_probs = torch.gather(probs, -1, shift_labels.unsqueeze(-1)).squeeze(-1)
    # Convert to log probabilities
    log_probs = actual_probs.log()

    for tok, log_probs in zip(tokens[1:], log_probs):
        print(f"{tok}: {log_probs:.6f}")

In [ ]:
get_logprobs_input(model, tokenizer, "")